<a href="https://colab.research.google.com/github/Condemor-bit/esmiprimeritodia/blob/main/Download%20%26%20subtitles%20with%20IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Entorno de ejecucion --->  cambiar tipo de entorno de ejeccucion --> seleccionar GPU**

In [ ]:
#@title Ejecutar esta celda para instalar las librería. // Press to Play
"""
this cell code is taken from Notebook = OpenAI Whisper - Adaptado por DotCSV https://www.youtube.com/watch?v=JuMEmF-2FsA
----
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356

"""

!pip install git+https://github.com/openai/whisper.git
!pip install jiwer
!pip install youtube-dl
!pip install googletrans==3.1.0a0

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import youtube_dl
from googletrans import Translator
from google.colab import drive
from google.colab import files

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
### for work with google drive remove # below //borra las siguientes # para trabajar en google drive
#drive.mount('/content/drive')  
#%cd drive/MyDrive/
#!ls

In [ ]:
#!youtube-dl -F  'URL LINK' #con este compruebas la formatos disponibles // check available formats

In [ ]:
!youtube-dl  -o videoname.mp4 -f best 'URL LINK' ## <----put video link here whitin ' ' // link del video entrecomillado ' '

In [ ]:
!whisper videoname.mp4 --task translate --model medium #se puede seguir con videos subidos manualmente desde aqui // if you upload manually a video, you can continue form here

In [ ]:
#@title Crea los sub en Spanish // se puede cambiat en des=' ' // can change the subtitles output in dest=' ' 
###============================Traduce el subtitulo=======================================================
with open('videoname.mp4.srt') as file:
    for line in file:
        paratra=(file.readlines())
jodernosale = [line for line in paratra if line.strip()]
traduccion=[]
for lines in jodernosale:
    translator=Translator()
#=========================================================================================================================================
    result = translator.translate(lines, src='en', dest='es')#### source= english (en) destination= spanish (es)
########## <--- change argument for translate into other language // cambia el argumento para poner otras lenguas (google translate)
#=========================================================================================================================================
    traduccion.append(result.text)
traduccion.insert(0, '1')
r = []
for i, v in enumerate(traduccion):
  r.append(v)
  if (i-2)%3==0:
    r.append("\n")
###crea y guarda el .srt 
with open('espanol.srt', mode='w') as file_object:
    for lineas in r:
        print(lineas, file=file_object)

In [ ]:
#========================================================================================================================================================================
 ##### add the name of subtitles created by whisper and google trans in subtitles= .srt // añade los subtitulos creados por whisper y google trans en subtitles= .srt 

In [ ]:
!ffmpeg -i videoname.mp4 -vf subtitles=nombrearchivo english_version.mp4 ### English version

In [ ]:
!ffmpeg -i videoname.mp4 -vf subtitles=espanol.srt spanish_version.mp4 ### Spanish version

In [ ]:
files.download('english_version.mp4') # Save english version

In [ ]:
files.download('spanish_version.mp4') # Save spanish version